# Guidance:
#### How to use the tool to generate XP metrics results:


1.   Activate the function - **test_result**
2.   Prepare the SQL query 
3.   Make sure the query returns the correct structure as requested
4.   Run the function on the SQL output 




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats
import math

from sqlalchemy import create_engine
from sqlalchemy import text

redshift_endpoint1 = "redshift.sayweee.net"
redshift_user1 = "jiafeng"
redshift_pass1 = "WEEEjiafeng2021!"
port1 = 5439 #whaterver your Redshift portnumber is
dbname1 = "dev"

engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user1, redshift_pass1, redshift_endpoint1, port1, dbname1)
engine1 = create_engine(engine_string)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


* '**ratio_ttest_2**' function: ratio-based test 
  * SQL output must return 4 columns: 
      * **col, variation, convert, total**
  * col: (optional) dimensions such as language, device_type, etc. Assign NULL / '' if dimension analysis is not needed.
  * variation: please return 1 & 2 as the result
  * convert: numerator 
  * total: denominator
* '**ttest**' function: Mean-based T-test 
  * SQL output must return 4 columns: 
      * **col, variation, id, metrics**
  * col: (optional) dimensions such as language, device_type, etc. Assign NULL / '' if dimension analysis is not needed.
  * variation: please return 1 & 2 as the result
  * id: granularity of the XP (device_id/user_id) 
  * metrics: returned values for metrics 

In [2]:
# ttest
def ttest(tb):
    tb = pd.DataFrame(tb.groupby(['variation','col']).describe().T)


    tb = tb.stack(level=[0,1])
    tb = pd.DataFrame(tb)
    tb = tb.reset_index(inplace=False)
    tb.rename(columns={'level_0': 'metrics', 'level_1': 'summary', 0: 'result'}, inplace=True)
    tb = tb.loc[tb['summary'].isin(['count', 'mean', 'std'])]
    tb.fillna(value=0, inplace=True)
    tb = tb.astype({"variation": str})
    tb = pd.merge(tb[tb["variation"] == '1'],tb[tb["variation"] == '2'],on=['col','summary','metrics'], how ='left')
    tb = pd.pivot_table(tb, index=['col', 'metrics'],columns=['summary'])
    # tb['Address'].str.split(',', expand=True)
    # tb.fillna(value=0, inplace=True)
    tb = tb.droplevel(0, axis=1)
    # tb = tb.stack()
    # tb = tb.reset_index(inplace=True)
    tb = tb.reset_index()
    tb.fillna(value=0, inplace=True)
    tb.columns = ['col','metrics', 'n_x','avg_x','stddev_x', 'n_y','avg_y','stddev_y']
    # calculate the t-statistic
    tb['t-stat'] = (tb['avg_x'] - tb['avg_y'])
    /
    np.sqrt((tb['stddev_x'])**2/tb['n_x']+
        (tb['stddev_y'])**2/tb['n_y'])
    
    # find the p-value & delta
    sig_conditions = [
        (tb['t-stat'] < -2.57),
        (tb['t-stat'] < -1.96) & (tb['t-stat'] > -2.57),
        (tb['t-stat'] < -1.645) & (tb['t-stat'] > -1.96),
        (tb['t-stat'] > 2.57),
        (tb['t-stat'] > 1.96) & (tb['t-stat'] <= 2.57),
        (tb['t-stat'] > 1.645) & (tb['t-stat'] < 1.96)
    ]
    sig_categories = [ '99% - negative', '95% - negative', '90% - negative','99% - positive', '95% - positive', '90% - positive']
    tb['significance'] = np.select(sig_conditions, sig_categories)
    tb['delta'] = (tb['avg_x'] / tb['avg_y']) - 1
    tb['delta'] = pd.Series(["{0:.2f}%".format(val * 100) for val in tb['delta']], index = tb.index)
    
    return tb

## ratio t-test function
from scipy.stats import ttest_ind
def ratio_ttest(click_a, num_a, click_b, num_b):
    a_dist = np.zeros(num_a)
    a_dist[:click_a] = 1
    b_dist = np.zeros(num_b)
    b_dist[:click_b] = 1
    zscore, prob = ttest_ind(a_dist, b_dist, equal_var=False)
    #r = print(f"Z-score is {zscore:0.2f}, p-value is {prob:0.3f} (two tailed), {prob/2:0.3f} (one tailed)")
    return prob


def ratio_ttest_2(tb):
  tb = pd.DataFrame(tb.groupby(['variation','col']).mean())
  # tb.reset_index(inplace=True)
  # tb = tb.droplevel(0, axis=1) 
  tb = tb.reset_index()
  tb = tb.astype({"variation": str, 'convert': int, 'total': int})
  tb = pd.merge(tb[tb["variation"] == '1'],tb[tb["variation"] == '2'],on=['col'], how ='left')
  # find the p-value & delta
      
  tb['p_value'] = 0
  tb['significance'] = 0
  for exp in range(len(tb)):
      click_a = tb.loc[exp, "convert_x"]
      num_a = tb.loc[exp, "total_x"]
      click_b = tb.loc[exp, "convert_y"]
      num_b = tb.loc[exp, "total_y"]
      tb.loc[exp, "p_value"] = ratio_ttest(click_a, num_a, click_b, num_b)


  sig_conditions = [
      (tb['p_value'] < 0.10) & (tb['p_value'] > 0.05),
      (tb['p_value'] <= 0.05) & (tb['p_value'] > 0.01),
      (tb['p_value'] < 0.01)
  ]
  sig_categories = ['90%', '95%', '99%']
  tb['significance'] = np.select(sig_conditions, sig_categories)
  tb['delta'] = (tb["convert_x"]/tb["total_x"]) / (tb["convert_y"]/tb["total_y"])-1
  tb['delta'] = pd.Series(["{0:.2f}%".format(val * 100) for val in tb['delta']], index = tb.index)
  return tb

# return the final datatable
import os
def test_result(df): 
  if 'total' in df.columns and 'convert' in df.columns and 'col' in df.columns and 'variation' in df.columns:
    return ratio_ttest_2(df)
  if len(df) > 2 and 'variation' in df.columns and 'col' in df.columns and 'stddev_x' in df.columns:
    return ttest(df)
  else:
    return "invalid SQL output, please follow the correct format"+ os.linesep + "mean-based: variation, col, XYZ, ABC"+ os.linesep + "ratio-based: variation, col, total, convert"

* '**ratio_ttest_2**' function: ratio-based test 






In [6]:
# example -ratio-ttest
sql3 = """
with exp_cvt as (
select u.visit_day,
u.CvT,
case when len(u.device_id)>=15 then 'A_' || u.device_id
when len(u.device_id)>1 then 'B_' || u.device_id else u.device_id end AS device_id,
l.device_lang
from (
select device_id,
visit_day,
case when variation = 1 then 1 else 2 end as CvT
from metrics.experiment_visitor
where experiment_id = 235
) u
left join
(
select
device_id, device_lang
from (
select device_id, LAST_VALUE(language) over(partition by device_id order by visit_day rows between unbounded preceding and unbounded following) device_lang
from metrics.experiment_visitor_sessions_details eusd
where experiment_id = 235 --and user_id = 4
) group by 1,2
) l
on u.device_id = l.device_id
group by 1,2,3,4
),
revenues as (
select --top 100
op.order_day,
op.device_id,
op.order_id,
sum(op.sub_total) as rev,
sum(op.quantity) as quantity,
sum(op.trade_in_discount) as trade_in_discount,
sum(op.gm_cost) as gm_cost,
sum(case when trade_in_discount > 0 then op.sub_total else 0 end) as trade_in_rev,
sum(case when trade_in_discount > 0 then op.gm_cost else 0 end) as trade_in_cost,
sum(case when trade_in_discount > 0 then op.quantity else 0 end) as trade_in_quantity
from metrics.order_product op
where op.order_day between '2022-04-07' and '2022-04-12'
group by 1,2,3
having trade_in_rev > 0
)

select
device_lang as col, 
e.CvT  as variation,
sum(rev) as total,
sum(gm_cost) as convert
from exp_cvt e
join revenues r
on e.device_id = r.device_id
and e.visit_day = r.order_day
where e.visit_day between '2022-04-07' and '2022-04-13'
group by 1,2

""" 
df1 = pd.read_sql_query(text(sql3), engine1)
ratio_ttest_2(df1)

,variation_x,col,total_x,convert_x,variation_y,total_y,convert_y,p_value,significance,delta
0,1,en,58356,38017,2,67055,43412,0.132989,0,0.63%
1,1,es,167,107,2,77,53,0.464358,0,-6.91%
2,1,ja,551,366,2,1312,835,0.249201,0,4.37%
3,1,ko,1785,1181,2,1736,1152,0.901672,0,-0.30%
4,1,vi,630,427,2,960,594,0.015496,95%,9.54%
5,1,zh,273513,176152,2,283370,183278,0.032329,95%,-0.42%
6,1,zh-Hant,34169,22202,2,41083,26626,0.633212,0,0.26%


* '**ttest**' function: Mean-based T-test 

In [5]:
# example - ttest
sql2 = """
with exp_cvt as (
select u.visit_day,
u.CvT,
case when len(u.device_id)>=15 then 'A_' || u.device_id
when len(u.device_id)>1 then 'B_' || u.device_id else u.device_id end AS device_id,
l.device_lang
from (
select device_id,
visit_day,
case when variation = 1 then 1 else 2 end as CvT
from metrics.experiment_visitor
where experiment_id = 235
) u
left join
(
select
device_id, device_lang
from (
select device_id, LAST_VALUE(language) over(partition by device_id order by visit_day rows between unbounded preceding and unbounded following) device_lang
from metrics.experiment_visitor_sessions_details eusd
where experiment_id = 235 --and user_id = 4
) group by 1,2
) l
on u.device_id = l.device_id
where device_lang in ('en','es')
group by 1,2,3,4
),
revenues as (
select --top 100
op.order_day,
op.device_id,
op.order_id,
sum(op.sub_total) as rev,
sum(op.quantity) as quantity,
sum(op.trade_in_discount) as trade_in_discount,
sum(op.gm_cost) as gm_cost,
sum(case when trade_in_discount > 0 then op.sub_total else 0 end) as trade_in_rev,
sum(case when trade_in_discount > 0 then op.gm_cost else 0 end) as trade_in_cost,
sum(case when trade_in_discount > 0 then op.quantity else 0 end) as trade_in_quantity
from metrics.order_product op
where op.order_day between '2022-04-07' and '2022-04-12'
group by 1,2,3
)

select
 device_lang as col, 
e.CvT as variation,
e.device_id as id,
--case when r.rev < 68 then '68-'
--else '68+' end as order_value_bucket,
sum(rev) as rev,
sum(quantity) as quantity,
sum(gm_cost) as gm_cost,
sum(trade_in_discount) as trade_in_discount,
sum(trade_in_rev) as trade_in_rev,
sum(trade_in_cost) as trade_in_cost,
sum(trade_in_quantity) as trade_in_quantity,
count(order_id) as order_cnt
from exp_cvt e
join revenues r
on e.device_id = r.device_id
and e.visit_day = r.order_day
where e.visit_day between '2022-04-07' and '2022-04-13'
group by 1,2,3

""" 
df_mean_based = pd.read_sql_query(text(sql2), engine1)
ttest(df_mean_based)

,col,metrics,n_x,avg_x,stddev_x,n_y,avg_y,stddev_y,t-stat,significance,delta
0,en,gm_cost,9763.0,42.703430,34.825496,9908.0,41.741194,32.925631,0.962236,0,2.31%
1,en,order_cnt,9763.0,1.175049,0.627010,9908.0,1.174102,0.650075,0.000947,0,0.08%
2,en,quantity,9763.0,14.288231,12.948406,9908.0,14.061365,10.557789,0.226867,0,1.61%
3,en,rev,9763.0,65.928947,52.265237,9908.0,64.547334,48.396463,1.381613,0,2.14%
4,en,trade_in_cost,9763.0,0.359537,2.070994,9908.0,0.498938,2.603361,-0.139401,0,-27.94%
5,en,trade_in_discount,9763.0,0.167982,0.864094,9908.0,0.286011,1.356938,-0.118029,0,-41.27%
6,en,trade_in_quantity,9763.0,0.136126,0.690920,9908.0,0.167945,0.780590,-0.031819,0,-18.95%
7,en,trade_in_rev,9763.0,0.594293,3.390693,9908.0,0.828323,4.338393,-0.234030,0,-28.25%
8,es,gm_cost,119.0,31.019160,38.862693,136.0,29.982059,26.287583,1.037101,0,3.46%
9,es,order_cnt,119.0,1.075630,0.348354,136.0,1.058824,0.292238,0.016807,0,1.59%


## Quick ratio-based p-value calculation

In [ ]:
welschs_ttest(1956,18196,1891,17948)

0.5102406445323776